In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

In [149]:
## load all data to a list - step_file_objs
# step_cal_dir = r'/data/old_trap/20220505/bead1/scan1/'

def extract_power_height(dirname):
    max_file = 1000
    power_array = []
    height_array = []
    freq_array = []
    for root, dirnames, filenames in os.walk(dirname):
        for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
#             print(dirname+filename)
            if '_fpga.h5' in filename[:]:
                try:
                    f = h5py.File(dirname+filename,'r')
                    dat = np.array(f['beads/data/pow_data'])
                    power_array.append(np.mean(dat[::4]))
                    height_array.append(f.attrs['bead_height'])
                    freq_array.append(extract_resonance(dirname+filename[:-8]+'.h5'))
                except:
                    print('***')
    power_array = np.array(power_array)
    height_array = np.array(height_array)
    freq_array = np.array(freq_array)
    
    return height_array, power_array, freq_array

def extract_resonance(filename):
    
    fsamp=5000
    res=50000
    df = bu.DataFile();
    df.load(filename);

    data=df.pos_data
    data_det=signal.detrend(data)
    xpsd, freqs = matplotlib.mlab.psd(data_det[0], Fs = fsamp, NFFT = res)
    psd_tmp = xpsd[100:]
    freq_tmp = freqs[100:]
    return freq_tmp[np.argmax(psd_tmp)]

In [152]:
step_cal_dir = r'/data/old_trap/20220512/bead1/scan2/'
height_array1, power_array1, freq_array1 = extract_power_height(step_cal_dir)

In [59]:
step_cal_dir = r'/data/old_trap/20220509/bead1/scan1/'
height_array2, power_array2, freq_array2 = extract_power_height(step_cal_dir)

In [60]:
power_array2

array([-287825.06624839, -287598.94759181])

In [64]:
-25000*10/2**15/10*50+50

11.85302734375

In [204]:
420e-12*8/1000*10, 10e-3/3e8, 5/80

(3.36e-11, 3.3333333333333335e-11, 0.0625)

In [115]:
0.069/350*1000*100*2, 1000*0.2/2/2

(39.42857142857143, 50.0)

In [644]:
np.log10(3.162)

0.49996186559619027

In [690]:
# step_cal_dir = r'/data/old_trap/20220520/bead1/scan7/'
# height_array1, power_array1, freq_array1 = extract_power_height(step_cal_dir)
# step_cal_dir = r'/data/old_trap/20220520/bead1/scan8/'
# height_array2, power_array2, freq_array2 = extract_power_height(step_cal_dir)
# step_cal_dir = r'/data/old_trap/20220520/bead1/scan9/'
# height_array3, power_array3, freq_array3 = extract_power_height(step_cal_dir)
# step_cal_dir = r'/data/old_trap/20220520/bead1/scan10/'
# height_array4, power_array4, freq_array4 = extract_power_height(step_cal_dir)
# step_cal_dir = r'/data/old_trap/20220520/bead1/scan11_reflected_pow/'
# height_array5, power_array5, freq_array5 = extract_power_height(step_cal_dir)
# step_cal_dir = r'/data/old_trap/20220520/bead1/scan12_reflected_pow/'
# height_array6, power_array6, freq_array6 = extract_power_height(step_cal_dir)

# step_cal_dir = r'/data/old_trap/20220525/bead1/scan1/'
# height_array5, power_array5, freq_array5 = extract_power_height(step_cal_dir)
step_cal_dir = r'/data/old_trap/20220525/bead1/scan2/'
height_array6, power_array6, freq_array6 = extract_power_height(step_cal_dir)

In [ ]:
reflectance_v = np.array([0.06, 1.12, 1.16, 0.56, 0.53, 0.52])
reflectance_pixel = np.array([-4.6, -8.44, -9.5, -3.45, -1.57, 0.12])
reflectance_mw = reflectance_v/30*2

In [ ]:
power_conversion = -1/222e3*46*0.666
power_conversion2 = 1/484e3*49.2
# power_conversion = -1
_,ax = plt.subplots(figsize=(9.2,5))
# ax.scatter(height_array1-4, power_array1*power_conversion, label = 'bead 1 - measurement 1')
# ax.scatter(height_array2, power_array2*power_conversion, label = 'bead 1 - measurement 2')
# ax.scatter(height_array3-7.9, power_array3*power_conversion, label = 'bead 1 - measurement 3')
# ax.scatter(height_array4, power_array4*power_conversion, label = 'bead 1 - measurement 4 - after the weekend')

# ax.scatter(height_array5, power_array5/power_array5[0]*30, label = 'bead 1 - measurement 5 - after the weekend')

ax.scatter(height_array5, power_array5*power_conversion2, label = 'bead 3 - measurement 1')
ax.scatter(height_array6, power_array6*power_conversion2, label = 'bead 3 - measurement 2')

# ax.scatter(height_array1, freq_array1, label = 'bead 1 - measurement 1')
ax.legend()
ax.set(ylim=(10,60), xlim=(-8,3.5), ylabel='Power [mW]', xlabel='Height [$\mu$m]')

In [396]:
power_conversion = -1/222e3*46*0.666
# power_conversion = -1
_,ax = plt.subplots(figsize=(9.2,5))
ax.scatter(reflectance_pixel, reflectance_mw, label = 'bead 1 - reflecatance - after the weekend')

# ax.scatter(height_array1, freq_array1, label = 'bead 1 - measurement 1')
ax.legend()
# ax.set(ylim=(10,35), xlim=(-25,120), ylabel='Power [mW]', xlabel='Height [$\mu$m]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [323]:
from skimage.registration import phase_cross_correlation as pcc
from skimage.io import imread as imread

import imageio as iio
from skimage import filters
from skimage.color import rgb2gray  # only needed for incorrectly saved images
from skimage.measure import regionprops

h_pcc = []
h_max = []
filename=r'/data/old_trap/20220520/bead1/height/20220520_4/run/image_test_0.bmp'
zeroth_image=imread(filename,as_gray=True)
for i in range(1,117):
    print(i,end=" ")
    filename=r'/data/old_trap/20220520/bead1/height/20220520_4/run/image_test_'+str(i)+'.bmp'
    image=imread(filename,as_gray=True)
    shift,_,_ = pcc(zeroth_image,image,upsample_factor=200)
    h_pcc.append(shift[0])


    threshold_value = filters.threshold_otsu(image)
    labeled_foreground = (image > threshold_value).astype(int)
    properties = regionprops(labeled_foreground, image)
    center_of_mass = properties[0].centroid
    h_max.append(230-center_of_mass[0])

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 

In [326]:
_,ax = plt.subplots()
ax.scatter(range(116), np.array(h_max)-h_max[0])
ax.scatter(range(116), np.array(h_pcc)-h_pcc[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [193]:
filename = r'/data/old_trap/20220512/bead1/scan2/10mbar_zcool_45.h5'
filename2 = r'/data/old_trap/20220512/bead1/scan2/10mbar_zcool_90.h5'
filename3 = r'/data/old_trap/20220512/bead1/scan2/10mbar_zcool_0.h5'

fsamp = 5000
res = 5000

df = bu.DataFile();
df.load(filename);

df2 = bu.DataFile();
df2.load(filename2);

df3 = bu.DataFile();
df3.load(filename3);

data=df.pos_data
data_det=signal.detrend(data)
xpsd, freqs = matplotlib.mlab.psd(data_det[0], Fs = fsamp, NFFT = res)
ypsd, freqs = matplotlib.mlab.psd(data_det[1], Fs = fsamp, NFFT = res)
zpsd, freqs = matplotlib.mlab.psd(data_det[2], Fs = fsamp, NFFT = res)
f = h5py.File(filename[:-3]+'_fpga.h5','r')
height1 = f.attrs['bead_height']

data2=df2.pos_data
data_det2=signal.detrend(data2)
xpsd2, freqs = matplotlib.mlab.psd(data_det2[0], Fs = fsamp, NFFT = res)
ypsd2, freqs = matplotlib.mlab.psd(data_det2[1], Fs = fsamp, NFFT = res)
zpsd2, freqs = matplotlib.mlab.psd(data_det2[2], Fs = fsamp, NFFT = res)
f = h5py.File(filename2[:-3]+'_fpga.h5','r')
height2 = f.attrs['bead_height']

data3=df3.pos_data
data_det3=signal.detrend(data3)
xpsd3, freqs = matplotlib.mlab.psd(data_det3[0], Fs = fsamp, NFFT = res)
ypsd3, freqs = matplotlib.mlab.psd(data_det3[1], Fs = fsamp, NFFT = res)
zpsd3, freqs = matplotlib.mlab.psd(data_det3[2], Fs = fsamp, NFFT = res)
f = h5py.File(filename3[:-3]+'_fpga.h5','r')
height3 = f.attrs['bead_height']

In [194]:
_,ax = plt.subplots()
ax.loglog(freqs, xpsd, '.-', label=str(height1*10)+' $\mu$m',markersize=1,linewidth=0.5)
ax.loglog(freqs, xpsd2, '.-', label=str(height2*10)+' $\mu$m',markersize=1,linewidth=0.5)
ax.loglog(freqs, xpsd3, '.-', label=str(height3*10)+' $\mu$m',markersize=1,linewidth=0.5)

ax.legend()
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel(r'PSD [bits$^2/Hz.$]')
# plt.xlim(1,600)
#plt.ylim(1e6,1e10)
plt.grid(True)

psd_tmp = xpsd[100:]
freq_tmp = freqs[100:]
freqs[np.argmax(xpsd)], freq_tmp[np.argmax(psd_tmp)]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 344.0)